# 0, 1 задание

## Декапитализация

Можно еще добавить имена, есть функция, которая находится наиболее частые:

In [1]:
import re
from collections import Counter

def extract_potential_names(text : str):
    # Предложение заканчивается на . ! ? или перевод строки, затем пробелы
    # Ищем слова с заглавной буквы не в начале предложения
    matches = re.findall(r'(?<![\.!\?\n]\s)(?<!^)\b[A-Z][a-z]+\b', text)
    return matches

def most_common_names(text : str, top_n=30):
    names = extract_potential_names(text)
    counter = Counter(names)
    return counter.most_common(top_n)

with open('Crime and Punishment ascii.txt', 'r', encoding='ascii') as f:
    original_text = f.read()

common_names = most_common_names(original_text, top_n=10)

print("Наиболее часто встречающиеся имена:")
for name, count in common_names:
    print(f"{name}: {count}")

Наиболее часто встречающиеся имена:
Raskolnikov: 408
Petrovitch: 272
Ivanovna: 268
Sonia: 256
Dounia: 234
Razumihin: 212
Katerina: 165
Porfiry: 138
Svidrigalov: 124
Pyotr: 123


В моей реализации это не добавлено

In [2]:
import re
import json
from collections import defaultdict

# Преобразование в битовый массив исключений 
def build_exception_bit_array(text: str) -> list[int]:
    bit_array = [0] * len(text)
    text_lower = text.lower()

    sentence_end = {'.', '!', '?', '\n'}

    i = 0
    while i < len(text):
        actual_char = text[i]
        expected_char = text_lower[i]  # по умолчанию — маленькая буква

        # --- Правило 1: первая буква текста ---
        if i == 0 and actual_char.isalpha():
            expected_char = actual_char.upper()

        # --- Правило 2: буква после .!? + любые пробелы/переводы строки ---
        if actual_char.isalpha():
            j = i - 1
            while j >= 0 and text[j] in {' ', '\n'}:
                j -= 1
            if j >= 0 and text[j] in sentence_end:
                expected_char = actual_char.upper()

        # --- Правило 3: после двух заглавных ---
        if i >= 2 and text[i - 2].isupper() and text[i - 1].isupper() and actual_char.isalpha():
            expected_char = actual_char.upper()

        # --- Правило 4: одинокая I между не-буквами ---
        if actual_char == 'i':
            left = i - 1
            right = i + 1
            while left >= 0 and not text[left].isalpha():
                left -= 1
            while right < len(text) and not text[right].isalpha():
                right += 1
            if (left < 0 or not text[left].isalpha()) and (right >= len(text) or not text[right].isalpha()):
                expected_char = 'I'

        # --- Сравнение: если не совпадает — исключение ---
        if actual_char != expected_char:
            bit_array[i] = 1

        i += 1

    return bit_array

# Обратное преобразование 
def restore_text_from_exceptions(text_lower: str, bit_array: list[int]) -> str:
    actual_text = []
    sentence_end = {'.', '!', '?', '\n'}

    i = 0
    while i < len(text_lower):
        actual_char = text_lower[i]

        # --- Правило 1: первая буква текста ---
        if i == 0:
            actual_char = actual_char.upper()

        # --- Правило 2: буква после .!? + любые пробелы/переводы строки ---
        if actual_char.isalpha():
            j = i - 1
            while j >= 0 and text_lower[j] in {' ', '\n'}:
                j -= 1
            if j >= 0 and text_lower[j] in sentence_end:
                actual_char = actual_char.upper()

        # --- Правило 3: после двух заглавных ---
        if i >= 2 and actual_text[i - 2].isupper() and actual_text[i - 1].isupper() and actual_char.isalpha():
            actual_char = actual_char.upper()

        # --- Правило 4: одинокая I между не-буквами ---
        if actual_char == 'i':
            left = i - 1
            right = i + 1
            while left >= 0 and not text_lower[left].isalpha():
                left -= 1
            while right < len(text_lower) and not text_lower[right].isalpha():
                right += 1
            if (left < 0 or not text_lower[left].isalpha()) and (right >= len(text_lower) or not text_lower[right].isalpha()):
                actual_char = actual_char.upper()

        # --- Меняем если это исключение ---
        if bit_array[i] == 1:  
            actual_char = actual_char.lower() if actual_char.isupper() else actual_char.upper()
        
        actual_text.append(actual_char)

        i += 1

    return ''.join(actual_text)


# Сохранение в json
def save_results(bit_array : list[int], rules_meta : dict, filename : str) -> None:
    output = {
        'bit_array': bit_array,
        'rules': rules_meta,
        'statistics': {
            'total_chars': len(bit_array),
            'exceptions': sum(bit_array),
            'compression_ratio': sum(bit_array) / len(bit_array) if len(bit_array) > 0 else 0
        }
    }

    with open(filename, 'w') as f:
        json.dump(output, f, indent=2)

In [3]:
file_path = "C:\\Users\\User\\Studies\\Кодирование\\CMDC\\task2\\Crime and Punishment ascii.txt"
# file_path = "C:\\Users\\User\\Studies\\Кодирование\\CMDC\\task2\\test1.txt"


with open(file_path, 'r') as f:
    content = f.read()

# Генерируем битовый массив
bit_array = build_exception_bit_array(content)

reconstructed = restore_text_from_exceptions(content.lower(), bit_array)

# Метаданные о правилах
rules_meta = {
    "standard_rules": [
        "First character of text",
        "After sentence-ending punctuation (.!?) followed by whitespace",
        "Third consecutive uppercase letter",
        "Lone 'I' between non-letters"
    ],
    "custom_rules": []
}

# Сохраняем все в json
# save_results(bit_array, rules_meta, "capitalization_rules.json")

# Проверка
print(content == reconstructed)

# print(json.dumps(content))
# print(json.dumps(reconstructed))

True


## Арифмитическое кодирование

### Первый способ

In [4]:
!pip install arithmetic-compressor

In [5]:
from arithmetic_compressor import AECompressor
from arithmetic_compressor.models import StaticModel

model = StaticModel({'0': 1125768, '1': 10717})
coder = AECompressor(model)

data = "".join(map(str, bit_array))

compressed = coder.compress(data)
decoded = coder.decompress(compressed, len(data))

print("Проверка декодера :", list(map(int, decoded)) == bit_array)
print("Доля полученного от исходного с помощью арифметического кодирования :", len(compressed) / len(data))

Проверка декодера : True
Доля полученного от исходного с помощью арифметического кодирования : 0.07698913756010858


### Второй способ

In [6]:
import math

# перерассчёт интервала с учётом новой частоты символов
def update_intervals(inter, count, alph, info_let, max_num):
  numAlph = len(alph)
  sumP = 0
  for i in range(numAlph-1):
    id = info_let[alph[i]][0]
    P = info_let[alph[i]][1] / count
    sumP += P
    high = math.ceil(sumP * max_num)

    inter[id][1] = high - 1
    inter[id+1][0] = high

# подсчёт повторяющихся первых бит
def find_common_leading_bits(num1, num2, N):
    common_bits = []
    for i in range(N):
        mask = (1 << (N - i - 1))
        bit1 = num1 & mask
        bit2 = num2 & mask
        if bit1 == bit2:
          if (bit1 > 0):
            common_bits.append("1")
          else:
            common_bits.append("0")
        else:
            return ''.join(common_bits), len(common_bits)

    return ''.join(common_bits), len(common_bits)

# подсчёт разницы границ интервалов
def check_interval_scaling(num1, num2, N):
    count = 0
    mask = (1 << (N - 1))
    if ((num1 & mask) != (num2 & mask)):
      for i in range(N-1):
        mask = (1 << (N - i - 2))
        bit1 = num1 & mask
        bit2 = num2 & mask
        if bit1 != 0 and bit2 == 0:
          count += 1
        else:
          return count

    return count

# создать список уникальных символов и словарь индексов букв
def create_alph_and_info(T):
    # Создаем список уникальных символов в порядке их первого появления
    alph = []
    for char in T:
        if char not in alph:
            alph.append(char)

    # Создаем словарь place
    info_let = {char: [index, 0] for index, char in enumerate(alph)}

    return alph, info_let

def print_interval_binary(low, high, bit_precision, label=""):
    """Выводит интервал в бинарном виде."""
    bin_low = bin(low)[2:].zfill(bit_precision)
    bin_high = bin(high)[2:].zfill(bit_precision)
    print(f"{label}: Low = {low}({bin_low}), High = {high}({bin_high})")


def arithmetic_encode(
    text,
    bit_precision,
    alphabet=None, 
    symbol_info=None,  
    static_model=False, 
    verbose=False
):  
    # изучение алфавита, если не передан
    if (alphabet == None):
        alphabet, symbol_info = create_alph_and_info(text)

    max_num = 2 ** bit_precision
    intervals = [[0, 0] for _ in range(len(alphabet))]  # Пример интервалов
    intervals[-1] = [0, max_num-1]
    
    # Маски для битовых операций
    full_mask = (1 << bit_precision) - 1
    lower_mask = (1 << (bit_precision - 1)) - 1
    highest_bit = 1 << (bit_precision - 1)

    # Инициализация частот символов
    total_symbols = 0
    if static_model:
        # Сброс и пересчет частот для статической модели
        for symbol in text:
            symbol_info[symbol][1] = 0
        for symbol in text:
            symbol_info[symbol][1] += 1
            total_symbols += 1
        update_intervals(intervals, total_symbols, alphabet, symbol_info, max_num)
    else:
        # Инициализация частот для адаптивной модели
        for symbol in alphabet:
            symbol_info[symbol][1] = 1
            total_symbols += 1

    # Начальные границы интервала
    low = 0
    high = max_num - 1
    pending_bits = 0
    encoded_bits = ""

    for symbol in text:
        if verbose:
            print(f"Символ: {symbol}")

        # Обновление интервалов для адаптивной модели
        if not static_model:
            update_intervals(intervals, total_symbols, alphabet, symbol_info, max_num)
            if verbose:
                print(f"Обновлённые интервалы: {intervals}")
            total_symbols += 1
            symbol_info[symbol][1] += 1

        # Вычисление нового интервала для символа
        symbol_id = symbol_info[symbol][0]
        interval_start, interval_end = intervals[symbol_id]
        range_size = high - low + 1

        new_low = low + math.ceil((range_size * interval_start) / max_num)
        new_high = low + math.ceil((range_size * (interval_end + 1)) / max_num) - 1

        if verbose:
            print_interval_binary(new_low, new_high, bit_precision, "Новый интервал")

        # Поиск общих старших битов
        common_bits, shift_amount = find_common_leading_bits(new_low, new_high, bit_precision)
        encoded_bits += common_bits

        if shift_amount > 0:
            # Добавление отложенных битов и сдвиг интервала
            encoded_bits += "1" * pending_bits
            pending_bits = 0

            new_low = (new_low << shift_amount) & full_mask
            new_high = ((new_high << shift_amount) | ((1 << shift_amount) - 1)) & full_mask

            if verbose:
                print_interval_binary(new_low, new_high, bit_precision, "Интервал после совпадающих битов")

        # Проверка на узкий интервал (E1, E2, E3)
        scale_bits = check_interval_scaling(new_low, new_high, bit_precision)
        if scale_bits > 0:
            # Масштабирование интервала
            saved_bit_low = new_low & highest_bit
            new_low = ((new_low << scale_bits) & lower_mask) | saved_bit_low

            saved_bit_high = new_high & highest_bit
            new_high = ((new_high << scale_bits) & lower_mask) | saved_bit_high | ((1 << scale_bits) - 1)

            if verbose:
                print_interval_binary(new_low, new_high, bit_precision, "Интервал после расширения")

        pending_bits += scale_bits

        if verbose:
            print(f"Закодированный текст: {encoded_bits}, Bits: {pending_bits}\n")

        low, high = new_low, new_high

    # if pending_bits > 0:
    #     encoded_bits += "1" * pending_bits

    return encoded_bits


text = "".join(map(str, bit_array))
result = arithmetic_encode(text, 8, alphabet=['0', '1'], symbol_info={'0': [0, 1125768], '1': [1, 10717]}, static_model=True, verbose=False)
print(len(text))
print("Доля полученного от исходного с помощью арифметического кодирования свой:", len(result) / len(text))

1136485
Доля полученного от исходного с помощью арифметического кодирования свой: 0.07987522932550803


# 2 задание

Я сохраняю контексты 3-го порядка следующим образом:

1) Сначала просто в виде contexts - это словарь, в котором хранится информация для каждого контекста
2) Далее для удобства преобразую этот словарь в сериализованный вид: [три символа, число символов после контекста = ns, ns раз : символ, его частота и так далее до конца]
3) Далее я кодирую статическим методом Хаффмана символы и троичным кодированием числа, учитывая структуру сериализованного массива

## Хранение контекстов 3-го порядка

In [7]:
from collections import defaultdict

# Строим контексты порядка order (по умолчанию 3)
def build_context_models(text : str, order=3) -> defaultdict(lambda: defaultdict(int)):
    contexts = defaultdict(lambda: defaultdict(int)) # Словрь из словарей

    for i in range(len(text) - order):
        context = text[i:i+order]
        next_char = text[i+order]
        contexts[context][next_char] += 1
    
    return contexts

# Считаем память, необходимую для хранения всех контекстов
def calculate_memory_usage_bytes(contexts : defaultdict(lambda: defaultdict(int))) -> int:
    # Расчёт памяти
    total_bytes = 0
    for context, chars in contexts.items():
        ns = len(chars)

        # Формула расчета : 
        # 1) 3 байта на символы в контексте
        # 2) 4 байта на хранение ns (int32)
        # 3) ns * 1 байт на хранение каждого символа после контекста
        # 4) ns * 4 байт на хранение частоты каждого символа (int32)
        total_bytes += 3 + 4 + ns * 1 + ns * 4 

    return total_bytes

file_path = "C:\\Users\\User\\Studies\\Кодирование\\CMDC\\task2\\Crime and Punishment ascii.txt"

with open(file_path, 'r') as f:
    content = f.read()

contexts = build_context_models(content.lower(), 3)
print(f"Объём памяти для хранения всех контекстов 3-го порядка: {calculate_memory_usage_bytes(contexts) / (1024**2):.2f} МБ")

Объём памяти для хранения всех контекстов 3-го порядка: 0.24 МБ


## Сериализация

Сделаем сериализацию, то есть чтобы все хранилось в одном массиве:
- 3 символа контекста (по одному в массиве)
- число ns (количество символов после данного контекста)
- 2 * ns раз идет : ['символ', 'его частота'] ...
- Новые 3 символа контекста и так далее  

In [8]:
contexts_expr = build_context_models(content.lower(), 0)
len(contexts_expr[''])

54

In [9]:
def serialize_model(contexts : defaultdict(lambda: defaultdict(int))):
    serialized = []
    sym_freqs = defaultdict(int)

    for ctx, d in contexts.items():
        c, t, x = list(ctx)
        serialized.extend(list(ctx))  # 3 символа контекста
        
        # для статического кода хаффмана считаем частоты
        sym_freqs[c] += 1
        sym_freqs[t] += 1
        sym_freqs[x] += 1

        serialized.append(len(d))     # количество продолжений

        for sym, freq in d.items():
            serialized.append(sym)
            serialized.append(freq)

            # для статического кода хаффмана считаем частоты
            sym_freqs[sym] += freq
        
    return serialized, sym_freqs

def calculate_memory_usage_serialized(serialized_model: list) -> int:
    total_bytes = 0
    i = 0
    while i < len(serialized_model):
        # 1. Контекст: 3 символа
        total_bytes += 3  # по 1 байту на символ
        i += 3

        # 2. Количество продолжений
        ns = serialized_model[i]
        total_bytes += 4  # как int32
        i += 1

        # 3. Каждое продолжение: символ (1 байт) + частота (4 байта)
        total_bytes += ns * (1 + 4)
        i += ns * 2  # символы и частоты
    return total_bytes

serialize_contexts, sym_freqs = serialize_model(contexts)
print(f"Объём памяти для хранения всех контекстов 3-го порядка сериализованная: {calculate_memory_usage_serialized(serialize_contexts) / (1024**2):.2f} МБ")

Объём памяти для хранения всех контекстов 3-го порядка сериализованная: 0.24 МБ


In [10]:
# пример сериализованного текста
print(contexts['cri'])
print(serialize_contexts[:20]) 

defaultdict(<class 'int'>, {'m': 101, 'e': 247, 'b': 35, 'p': 11, 'f': 15, 'n': 4, 's': 8, 't': 8})
['c', 'r', 'i', 8, 'm', 101, 'e', 247, 'b', 35, 'p', 11, 'f', 15, 'n', 4, 's', 8, 't', 8]


In [11]:
# Таблица частот для статического кода Хаффмана
sym_freqs

defaultdict(int,
            {'c': 19404,
             'r': 49270,
             'i': 63556,
             'm': 22997,
             'e': 106137,
             'b': 12699,
             'p': 14542,
             'f': 17658,
             'n': 63835,
             's': 54324,
             't': 81417,
             'l': 36684,
             'y': 21930,
             ' ': 189494,
             'a': 75170,
             '\n': 23615,
             ',': 16515,
             '.': 16587,
             ';': 1274,
             'd': 39768,
             '!': 2672,
             '-': 2931,
             '?': 2560,
             'v': 11427,
             'o': 73224,
             'h': 56413,
             'u': 28571,
             'w': 21398,
             'k': 10208,
             'g': 18917,
             'q': 862,
             'j': 951,
             '(': 293,
             '_': 885,
             'x': 1487,
             ')': 361,
             ':': 380,
             'z': 1267,
             '4': 36,
             '*': 58,
    

## Кодирование исходного списка

### Дерево для статического кода Хаффана по символам

In [12]:
import heapq
from collections import Counter

class HuffmanNode:
    def __init__(self, freq, symbol=None, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(freqs):
    heap = [HuffmanNode(freq, symbol) for symbol, freq in freqs.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        parent = HuffmanNode(left.freq + right.freq, None, left, right)
        heapq.heappush(heap, parent)

    return heap[0]

def build_code_table(node, prefix='', table=None):
    if table is None:
        table = {}
    if node.symbol is not None:  # Лист
        table[node.symbol] = prefix
    else:
        build_code_table(node.left, prefix + '0', table)
        build_code_table(node.right, prefix + '1', table)
    return table

In [13]:
freqs = sym_freqs

tree = build_huffman_tree(freqs)
code_table = build_code_table(tree)

print("Символы и коды:")
for k, v in code_table.items():
    print(f"{k!r}: {v}")

bit_count = 0
for k, v in code_table.items():
    bit_count += len(v) + 8 # на хранение символа 8 бит + длина кода

print("Размер дерева :", bit_count)

Символы и коды:
'e': 000
's': 0010
'h': 0011
'u': 01000
'-': 01001000
'(': 01001001000
')': 01001001001
'q': 0100100101
'_': 0100100110
':': 01001001110
'1': 0100100111100
'0': 01001001111010
'*': 01001001111011
'6': 010010011111000
'3': 010010011111001
'[': 0100100111110100
'7': 0100100111110101
'8': 010010011111011
'4': 010010011111100
'5': 010010011111101
'/': 0100100111111100
'9': 0100100111111101
'2': 0100100111111110
']': 01001001111111110
'%': 010010011111111110
'$': 010010011111111111
'k': 0100101
',': 010011
'i': 0101
'n': 0110
'.': 011100
'f': 011101
'l': 01111
'o': 1000
'a': 1001
'g': 101000
'c': 101001
'd': 10101
't': 1011
'w': 110000
'j': 1100010000
'z': 1100010001
'?': 110001001
'!': 110001010
';': 1100010110
'x': 1100010111
'v': 1100011
'y': 110010
'm': 110011
'r': 11010
'\n': 110110
'b': 1101110
'p': 1101111
' ': 111
Размер дерева : 936


### Троичное кодирование для числовых форматов

In [14]:
# Перевод в троичную систему
def to_ternary(n: int) -> list[int]:
    digits = []
    while n > 0:
        digits.append(n % 3)
        n //= 3
    return digits[::-1]

# Троичное кодирование числа n
def encode_number_ternary(n: int) -> str:
    if n <= 0:
        raise ValueError("n <= 0 Ternary encoder")
    
    digits = to_ternary(n)
    code = format(digits[0] - 1, '01b') # старшая цифра

    for d in digits[1:]:
        code += format(d, '02b') # оставшиеся

    code += '11' # запятая

    return code

# Декодирует троичное число из строки, возвращает (число, позиция после чтения)
def decode_number_ternary(bitstring: str, start=0):
    i = start
    # Читаем старшую цифру
    ik = int(bitstring[i]) + 1
    i += 1
    digits = [ik]
    while i + 1 < len(bitstring):
        
        if bitstring[i:i+2] == '11': # Смотрим на запятую
            i += 2
            break
        
        digits.append(int(bitstring[i:i+2], 2))
        i += 2
    
    # Восстанавливаем число 
    n = 0
    for d in digits:
        n = n * 3 + d
    
    return n, i

### Гамма код Элиаса для числового формата

In [15]:
from math import floor, log2

# Кодирует положительное целое число n с помощью кода Элиаса γ
def encode_number_elias_gamma(n: int) -> str:
    if n <= 0:
        raise ValueError("n <= 0 gamma Eliase code")
    binary = bin(n)[2:]  # двоичное представление без '0b'
    length = len(binary)
    prefix = '0' * (length - 1)
    return prefix + binary

# Декодирует число из строки с помощью кода Элиаса гамма
def decode_number_elias_gamma(bitstring: str, start=0):
    i = start
    zero_count = 0

    while i < len(bitstring) and bitstring[i] == '0':
        zero_count += 1
        i += 1
    
    if i + zero_count >= len(bitstring):
        raise ValueError("Problem")
    
    binary = bitstring[i:i + zero_count + 1]
    i += zero_count + 1
    
    return int(binary, 2), i

### Исходный алгоритм кодирования

In [ ]:
# sequence: список структур: [ctx0, ctx1, ctx2, n, [sym0, freq0, sym1, freq1, ...]]
# symbol_to_code: словарь символов -> битовая строка Хаффмана
# number_encoder: функция, например, encode_number_ternary или encode_number_elias_gamma
def encode_model_sequence(sequence, symbol_to_code, number_encoder):
    encoded = ''
    i = 0

    while i < len(sequence):
        
        # 3 символа контекста
        for j in range(3):
            encoded += symbol_to_code[sequence[i]]
            i += 1

        # Число продолжений
        n = sequence[i]
        i += 1
        encoded += number_encoder(n)

        # Символ + число n раз
        for _ in range(n):
            sym = sequence[i]
            freq = sequence[i + 1]
            i += 2
            encoded += symbol_to_code[sym]
            encoded += number_encoder(freq)
    
    return encoded

# bitstring: строка битов
# code_to_symbol: dict, где ключ — путь по дереву, значение — символ
# number_encoder: функция, например, encode_number_ternary или encode_number_elias_gamma
def decode_model_sequence(bitstring, code_to_symbol, number_decoder):
    i = 0
    result = []
    
    # Метод ля поиска символа по дереву
    def read_symbol():
        nonlocal i
        path = ''
        while i < len(bitstring):
            path += bitstring[i]
            i += 1
            if path in code_to_symbol:
                return code_to_symbol[path]
        raise ValueError("Symbol decode failed.")

    while i < len(bitstring):
        ctx = [read_symbol() for _ in range(3)]
        result.extend(ctx)
        n, i = number_decoder(bitstring, i)
        result.append(n)
        
        for _ in range(n):
            sym = read_symbol()
            freq, i = number_decoder(bitstring, i)
            result.extend([sym, freq])
    
    return result

### Проверка алгоритмов и выбор лучшего

In [ ]:
symbol_to_code = code_table
code_to_symbol = {v: k for k, v in symbol_to_code.items()}

# Троичное кодирование
encoded_ternary = encode_model_sequence(serialize_contexts, symbol_to_code, encode_number_ternary)
decoded_ternary = decode_model_sequence(encoded_ternary, code_to_symbol, decode_number_ternary)

# гамма Элиас
encoded_gamma = encode_model_sequence(serialize_contexts, symbol_to_code, encode_number_elias_gamma)
decoded_gamma = decode_model_sequence(encoded_gamma, code_to_symbol, decode_number_elias_gamma)

print("Length (ternary):", len(encoded_ternary))
print("Length (Eliase gamma):", len(encoded_gamma))
print(decoded_ternary == serialize_contexts)
print(decoded_gamma == serialize_contexts)

Length (ternary): 587995
Length (Eliase gamma): 581041
True
True
